In [2]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
sub = pd.read_csv("../input/sample_submission.csv")
structure = pd.read_csv("../input/structures.csv")

In [4]:
train.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548


In [5]:
test.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type
0,4659076,dsgdb9nsd_000004,2,0,2JHC
1,4659077,dsgdb9nsd_000004,2,1,1JHC
2,4659078,dsgdb9nsd_000004,2,3,3JHH


In [6]:
structure.head(3)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277


In [7]:
train_atom0 = pd.merge(
    train, 
    structure,
    left_on=["molecule_name", "atom_index_0"],
    right_on=["molecule_name", "atom_index"],
    how="left"
)

train_atom0.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_index,atom,x,y,z
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,1,H,0.00215,-0.006031,0.001976
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,1,H,0.00215,-0.006031,0.001976
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,1,H,0.00215,-0.006031,0.001976


In [8]:
train_atom0 = train_atom0.drop("atom_index", axis=1)
train_atom0 = train_atom0.rename(
    columns={
        "atom": "atom_0",
        "x": "x_0",
        "y": "y_0",
        "z": "z_0",
    }
)

train_atom0.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.00215,-0.006031,0.001976
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.00215,-0.006031,0.001976


In [9]:
train_atom0_atom1 = pd.merge(
    train_atom0, 
    structure,
    left_on=["molecule_name", "atom_index_1"],
    right_on=["molecule_name", "atom_index"],
    how="left"
)

train_atom0_atom1.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_index,atom,x,y,z
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,0,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.00215,-0.006031,0.001976,2,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.00215,-0.006031,0.001976,3,H,-0.540815,1.447527,-0.876644


In [10]:
train_atom0_atom1 = train_atom0_atom1.drop("atom_index", axis=1)
train_atom0_atom1 = train_atom0_atom1.rename(
    columns={
        "atom": "atom_1",
        "x": "x_1",
        "y": "y_1",
        "z": "z_1",
    }
)

train_atom0_atom1.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.00215,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.00215,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644


In [11]:
train = train_atom0_atom1

In [12]:
test_atom0 = pd.merge(
    test, 
    structure,
    left_on=["molecule_name", "atom_index_0"],
    right_on=["molecule_name", "atom_index"],
    how="left"
)

test_atom0.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,atom_index,atom,x,y,z
0,4659076,dsgdb9nsd_000004,2,0,2JHC,2,H,-1.661639,0.0,1.0
1,4659077,dsgdb9nsd_000004,2,1,1JHC,2,H,-1.661639,0.0,1.0
2,4659078,dsgdb9nsd_000004,2,3,3JHH,2,H,-1.661639,0.0,1.0


In [13]:
test_atom0 = test_atom0.drop("atom_index", axis=1)
test_atom0 = test_atom0.rename(
    columns={
        "atom": "atom_0",
        "x": "x_0",
        "y": "y_0",
        "z": "z_0",
    }
)

test_atom0.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0
0,4659076,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0
1,4659077,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0
2,4659078,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0


In [14]:
test_atom0_atom1 = pd.merge(
    test_atom0, 
    structure,
    left_on=["molecule_name", "atom_index_1"],
    right_on=["molecule_name", "atom_index"],
    how="left"
)

test_atom0_atom1.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_index,atom,x,y,z
0,4659076,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,0,C,0.599539,0.0,1.0
1,4659077,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,1,C,-0.599539,0.0,1.0
2,4659078,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,3,H,1.661639,0.0,1.0


In [15]:
test_atom0_atom1 = test_atom0_atom1.drop("atom_index", axis=1)
test_atom0_atom1 = test_atom0_atom1.rename(
    columns={
        "atom": "atom_1",
        "x": "x_1",
        "y": "y_1",
        "z": "z_1",
    }
)

test_atom0_atom1.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4659076,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4659077,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4659078,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0


In [16]:
test = test_atom0_atom1

In [17]:
import numpy as np

train_p_0 = train[["x_0", "y_0", "z_0"]].values
train_p_1 = train[["x_1", "y_1", "z_1"]].values
test_p_0 = test[["x_0", "y_0", "z_0"]].values
test_p_1 = test[["x_1", "y_1", "z_1"]].values

train["dist"] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
train["dist_x"] = np.square(train["x_0"] - train["x_1"])
train["dist_y"] = np.square(train["y_0"] - train["y_1"])
train["dist_z"] = np.square(train["z_0"] - train["z_1"])

test["dist"] = np.linalg.norm(test_p_0 - test_p_1, axis=1)
test["dist_x"] = np.square(test["x_0"] - test["x_1"])
test["dist_y"] = np.square(test["y_0"] - test["y_1"])
test["dist_z"] = np.square(test["z_0"] - test["z_1"])


In [18]:
train.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953,0.000220,1.192105,0.000036
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.00215,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120,1.019253,2.160261,0.000003
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.00215,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147,0.294812,2.112831,0.771973


In [19]:
train["type_0"] = train["type"].apply(lambda x: x[0])
test["type_0"] = test["type"].apply(lambda x: x[0])

In [20]:
from tqdm import tqdm_notebook

def create_features(df):
    df["molecule_couples"] = df.groupby("molecule_name")["id"].transform("count")
    
    df["molecule_dist_mean"] = df.groupby("molecule_name")["dist"].transform("mean")
    df["molecule_dist_min"] = df.groupby("molecule_name")["dist"].transform("min")
    df["molecule_dist_max"] = df.groupby("molecule_name")["dist"].transform("max")
    df["molecule_dist_std"] = df.groupby("molecule_name")["dist"].transform("std")
    
    df["atom_0_couples"] = df.groupby(["molecule_name", "atom_index_0"])["id"].transform("count")
    df["atom_1_couples"] = df.groupby(["molecule_name", "atom_index_1"])["id"].transform("count")
    
    categorical_columns = ["atom_index_0", "atom_index_1", "type", "type_0", "atom_0", "atom_1"]
    numerical_columns = ["x_0", "y_0", "z_0", "x_1", "y_1", "z_1", "dist", "dist_x", "dist_y", "dist_z"]
    stat_values = ["mean", "min", "max", "std"]
    
    for column in categorical_columns:
        df[f"molecule_{column}_count"] = df.groupby("molecule_name")[column].transform("count")
    
    for categorical_column in tqdm_notebook(categorical_columns):
        for numerical_column in numerical_columns:
            for stat_value in stat_values:
                df[f"molecule_{categorical_column}_{numerical_column}_{stat_value}"] = df.groupby(["molecule_name", categorical_column])[numerical_column].transform(stat_value)
                df[f"molecule_{categorical_column}_{numerical_column}_{stat_value}_diff"] = df[f"molecule_{categorical_column}_{numerical_column}_{stat_value}"] - df[numerical_column]
                df[f"molecule_{categorical_column}_{numerical_column}_{stat_value}_div"] = df[f"molecule_{categorical_column}_{numerical_column}_{stat_value}"] / df[numerical_column]
    
    return df
    

In [21]:
train = create_features(train)

In [23]:
print(train.shape)
train.head(3)

(4659076, 752)


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,molecule_atom_1_dist_z_mean_div,molecule_atom_1_dist_z_min,molecule_atom_1_dist_z_min_diff,molecule_atom_1_dist_z_min_div,molecule_atom_1_dist_z_max,molecule_atom_1_dist_z_max_diff,molecule_atom_1_dist_z_max_div,molecule_atom_1_dist_z_std,molecule_atom_1_dist_z_std_diff,molecule_atom_1_dist_z_std_div
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,...,10949.356227,0.000036,0.00000,1.000000,0.807116,0.807080,2.223513e+04,0.458992,0.458956,12644.708608
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.00215,-0.006031,0.001976,...,366933.880758,0.000003,0.00000,1.000000,3.179235,3.179232,1.100670e+06,1.086100,1.086097,376014.208780
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.00215,-0.006031,0.001976,...,1.372940,0.000003,-0.77197,0.000004,3.179235,2.407262,4.118325e+00,1.086100,0.314127,1.406915


In [22]:
test = create_features(test)

In [24]:
test.shape

(2505190, 751)

In [25]:
train.to_csv("../input/bf_train.csv", index=False)
test.to_csv("../input/bf_test.csv", index=False)

In [ ]:
train = pd.read_csv("../input/bf_train.csv")
test = pd.read_csv("../input/bf_test.csv")

In [ ]:
featColumns = [
    "molecule_atom_index_0_dist_min",
    "molecule_atom_index_0_dist_max",
    "molecule_atom_index_1_dist_min",
    "molecule_atom_index_0_dist_mean",
    "molecule_atom_index_0_dist_std",
    "dist",
    "molecule_atom_index_1_dist_std",
    "molecule_atom_index_1_dist_max",
    "molecule_atom_index_1_dist_mean",
    "molecule_atom_index_0_dist_max_diff",
    "molecule_atom_index_0_dist_max_div",
    "molecule_atom_index_0_dist_std_diff",
    "molecule_atom_index_0_dist_std_div",
    "atom_0_couples_count",
    "molecule_atom_index_0_dist_min_div",
    "molecule_atom_index_1_dist_std_diff",
    "molecule_atom_index_0_dist_mean_div",
    "atom_1_couples_count",
    "molecule_atom_index_0_dist_mean_diff",
    "molecule_couples",
    "atom_index_1",
    "molecule_dist_mean",
    "molecule_atom_index_1_dist_max_diff",
    "molecule_atom_index_0_y_1_std",
    "molecule_atom_index_1_dist_mean_diff",
    "molecule_atom_index_1_dist_std_div",
    "molecule_atom_index_1_dist_mean_div",
    "molecule_atom_index_1_dist_min_diff",
    "molecule_atom_index_1_dist_min_div",
    "molecule_atom_index_1_dist_max_div",
    "molecule_atom_index_0_z_1_std",
    "y_0",
    "molecule_type_dist_std_diff",
    "molecule_atom_1_dist_min_diff",
    "molecule_atom_index_0_x_1_std",
    "molecule_dist_min",
    "molecule_atom_index_0_dist_min_diff",
    "molecule_atom_index_0_y_1_mean_diff",
    "molecule_type_dist_min",
    "molecule_atom_1_dist_min_div",
    "atom_index_0",
    "molecule_dist_max",
    "molecule_atom_1_dist_std_diff",
    "molecule_type_dist_max",
    "molecule_atom_index_0_y_1_max_diff",
    "molecule_type_0_dist_std_diff",
    "molecule_type_dist_mean_diff",
    "molecule_atom_1_dist_mean",
    "molecule_atom_index_0_y_1_mean_div",
    "molecule_type_dist_mean_div",
    "type",
]

In [ ]:
train["type"].unique()

In [ ]:
import seaborn as sns

for idx, val in enumerate(train["type"].unique()):
    sns.scatterplot(
        data=train, 
        x=train.loc[train["type"] == val, featColumns[0]], 
        y=train.loc[train["type"] == val, "scalar_coupling_constant"],
    )